In [16]:
import torch
import numpy as np

In [2]:
!pip install datasets transformers huggingface_hub



In [3]:
from datasets import load_dataset


In [4]:
imdb = load_dataset("generics_kb", name = 'generics_kb')



Found cached dataset generics_kb (/Users/nathanielalexis/.cache/huggingface/datasets/generics_kb/generics_kb/1.0.0/9b41cde494db24f842a9260588bcfb2e3a257364568666ef240e98c70fb0e709)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
#Code to generate Yoda Text from https://github.com/yevbar/Yoda-Script/blob/master/yoda.py
import spacy
punctuation = [',', '.', ';', '?']
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
comma = nlp('Hello, World')[1]

def sentify(text):
    output = []
    doc = nlp(text)
    for sent in doc.sents:
        sentence = []
        end_punctuation = sent[-1]
        for clause in clausify(sent[:-1]):
            sentence.append(yodafy(clause))
        sentence[-1].append(end_punctuation)
        output.append(sentence)
    return output

def clausify(sent):
    output = []
    cur = []

    prev_token = None
    for token in sent:
        if prev_token!= None and prev_token.dep_ == 'punct' and prev_token.text in punctuation and token.dep_ != "amod":
            #(token.dep_ == 'cc' or (token.dep_ == 'punct' and token.text in punctuation)):
            output.append(cur)
            #output.append([token])
            cur = [token]
        else:
            cur.append(token)
        prev_token = token
    if cur != []:
        output.append(cur)
    return output

def yodafy(clause):
    new_array = []
    state = False
    prev_token = None
    flag = False
    for token in clause:
        if state:
            new_array.append(token)
        elif not state and prev_token != None and (prev_token.dep_ == "ROOT" or prev_token.dep_ == "aux") and token.dep_ != "nsubj" or flag:
            state = True
            new_array.append(token)
        elif not state and prev_token != None and (prev_token.dep_ == "ROOT" or prev_token.dep_ == "aux"):
            flag = True
        prev_token = token
    if len(new_array) > 0 and new_array[len(new_array)-1].dep_ != 'punct':
        new_array.append(comma)
    prev_token = None
    for token in clause:
        new_array.append(token)
        if (token.dep_ == "ROOT" or token.dep_ == "aux") and flag == False:
            break
        elif prev_token != None and (prev_token.dep_ == "ROOT" or prev_token.dep_ == "aux") and flag:
            break
        prev_token = token
    return new_array

def yoda(string_):
    string = []
    #end_punctuation = string_[-1]
    yodafied = sentify(string_)
    for sentence in yodafied:
        sentence_ = ""
        for clause in sentence:
            for token in clause:
                if token.dep_ == 'NNP' or token.dep_ == 'NNPS' or token.text == 'I':
                    sentence_ += token.text + " "
                elif sentence_ == "" and token.dep_ == 'neg':
                    sentence_ += "Not" + " "
                elif sentence_ == "":
                    sentence_ += token.text[0].upper() + token.text[1:] + " "
                elif token.dep_ == 'punct':
                    sentence_ = sentence_[:len(sentence_)-1] + token.text + " "
                else:
                    sentence_+=token.text.lower() + " "
        string.append(sentence_)
    return "".join(string) #+ end_punctuation

In [6]:
yoda("AA batteries maintain the settings if the power ever goes off.")

'The settings if the power ever goes off, aa batteries maintain. '

In [7]:
print(imdb['train'][38994]['generic_sentence'])
print(imdb['train'][38994].keys())

Additional support is provided through merchandising, group tours and affinity programs.
dict_keys(['source', 'term', 'quantifier_frequency', 'quantifier_number', 'generic_sentence', 'score'])


In [17]:
print(len(imdb['train']))

3433000


In [19]:
arr = []
indices = np.random.randint(0, len(imdb['train'])-1, size=50000)
for i in indices:
    arr.append(imdb['train'][int(i)]['generic_sentence'])

In [20]:
print(arr[:10])

['Degree requirements Thirty-two credits are required for the degree.', 'Issue permits for plants which break and can eggs.', 'Earlier submission is encouraged and appreciated.', 'War simply increases the hardship on children and families struggling to gain basic foods.', 'Participants can walk or run as individuals or in teams.', 'Most people pride themselves on their ability to fit in with others.', 'All systems fall on a continuum between the two.', 'Posts there get to few readers.', 'Greens are fast and true.', 'Students design their flexible clusters of expertise in consultation with their tutors.']


In [21]:
print(".\n.")

.
.


In [130]:
import data_funcs as df
import importlib
importlib.reload(df)

<module 'data_funcs' from '/home/webs8328/cs182_proj/data/data_funcs.py'>

In [115]:
h = df.get_dataset()

In [120]:
print(h)

Dataset({
    features: ['source', 'term', 'quantifier_frequency', 'quantifier_number', 'generic_sentence', 'score'],
    num_rows: 3433000
})


In [131]:
x = df.format_data_yoda(h, [1, 9, 200])
for i in x:
    print(i)

User: Tell me about a.net income.
Yoda: By net sales, a.net income is divided. 
User: Tell me about aa battery.
Yoda: The settings if the power ever goes off, aa batteries maintain. 
User: Tell me about aardvark.
Yoda: An amount of short coarse hair, aardvarks only have. 
